# average per capita by age and percentage of people by age and vantage mortgage who EVER have had a mortgage

What do the average per capita mortgage levels would be once you adjust for the joint mortgage issue. this could be different for you vs old people since they have different likelihood of being in a relationship. just take the observed likelihood that people in the different age groups hold mortgages jointly or individually.

In [1]:
# Setting up 

import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

sample = "~//projects//equifaxmacro_proj//EquiFax3//2_10_Percent_Sample_Construction//output//sample//"
output_3 = "~//projects//equifaxmacro_proj//EquiFax3//3_Balance_In_And_Out_Construction//output//"

In [169]:
# read in a month
cols = ['consumer_id', 'trade_id', 'product_category', 'balance', 'ecoa', 'consumer_age_archive']

month_str = sample + "202001_10perc.parquet"
mon = pd.read_parquet(month_str, columns = cols)

In [170]:
# only keep first mortgages
total_ppl = mon.groupby('consumer_age_archive').apply(lambda group: len(group.consumer_id.unique()))
mon = mon[mon.product_category == "FM"]

total_mort_ppl = mon.groupby('consumer_age_archive').apply(lambda group: len(group.consumer_id.unique()))

In [160]:
mon.head()

,consumer_id,trade_id,product_category,balance,ecoa,consumer_age_archive
4,23291220,1031986843,FM,327820,I,3
939,103838,1032019573,FM,189463,I,3
1393,14973534,1029445310,FM,241139,J,2
1394,21395140,1029445310,FM,241139,J,2
1400,10826332,1032043676,FM,195921,I,2


In [161]:
mon.shape

(811985, 6)

In [162]:
# we want to know the percentage of people that have at least one joint mortgage. Let's figure out the number of people first



In [163]:
# now let's do the same, but filter out all the people without joint mortgages

joint = mon[mon.ecoa=="J"]

In [164]:
joint_ppl = joint.groupby('consumer_age_archive').apply(lambda group: len(group.consumer_id.unique()))

consumer_age_archive
0    0.669550
1    0.564372
2    0.599753
3    0.639210
4    0.653657
5    0.666171
6    0.681523
7    0.650927
dtype: float64

now, we want to adjust the per mortgage balance per capita 
From Antoinette: "just take the observed likelihood that people in the different age groups hold mortgages jointly or individually."

Isn't the best option still to just drop half of all the joint tradlines? 

In [148]:
mon.head()

,consumer_id,trade_id,product_category,balance,ecoa,consumer_age_archive
4,23291220,1031986843,FM,327820,I,3
939,103838,1032019573,FM,189463,I,3
1393,14973534,1029445310,FM,241139,J,2
1394,21395140,1029445310,FM,241139,J,2
1400,10826332,1032043676,FM,195921,I,2


In [149]:
# get all the mortgages again
# drop half of all the joint mortgages 
# figure out the balances per capita (by state?)
mort = mon[mon.product_category == "FM"]

# separate individual and joint
ind = mort[mort.ecoa == "I"]
joint = mort[mort.ecoa == "J"]

In [150]:
# uhhhh, I guess we just drop half the tradelines 

joint.head()

,consumer_id,trade_id,product_category,balance,ecoa,consumer_age_archive
1393,14973534,1029445310,FM,241139,J,2
1394,21395140,1029445310,FM,241139,J,2
1444,17189636,717601721,FM,238661,J,5
1469,22144587,929564376,FM,90094,J,4
1621,26169177,717672696,FM,141519,J,3


In [151]:
# okay, so we can just drop half the tradelines directly? 
joint.shape

(521631, 6)

In [152]:
joint = joint.iloc[0::2,:]
joint.shape

(260816, 6)

In [153]:
new_mort = pd.concat([ind, joint])

In [154]:
# this is balance 

new_mort.groupby('consumer_age_archive').sum()['balance']

consumer_age_archive
0      227305532
1      504807271
2    11521172388
3    23981317551
4    24795232805
5    19699044695
6    10656441240
7     3714570199
Name: balance, dtype: int64

In [155]:
total_ppl

consumer_age_archive
0      5332
1    178411
2    398606
3    383249
4    376947
5    394799
6    309516
7    242970
dtype: int64

In [156]:
# this is balance 

new_mort.groupby('consumer_age_archive').sum()['balance']/total_ppl

consumer_age_archive
0    42630.444861
1     2829.462707
2    28903.660226
3    62573.725048
4    65779.095748
5    49896.389543
6    34429.371147
7    15288.184545
dtype: float64

In [157]:
new_mort['balance'].sum()/total_ppl.sum()

41531.42009712511

In [180]:
mort['balance'].sum()/total_ppl.sum()

63463.98103702022

In [184]:
finalstat = pd.DataFrame([new_mort['balance'].sum()/total_ppl.sum(), joint_ppl.sum()/total_mort_ppl.sum()])
finalstat.index = ['all_mort_per_capita', 'all_perc_joint']
finalstat

,0
all_mort_per_capita,41531.420097
all_perc_joint,0.650321


In [166]:
# okay, so we want two things: 
# 1) percentage of mortgages that are joint mortgages by age? And overall? 
# 2) percentage of people with a mortgage by age (and overall)
# 3) mortgage per capita, by age and overall 

In [173]:
perc_mort = joint_ppl/total_mort_ppl
perc_mort.index = ['Unknown',
                    '18–24',
                    '25–34',
                    '35-44',
                    '45-54',
                    '55–64',
                    '65–74',
                    '75+'] 

perc_mort.index = perc_mort.index + "_mort_perc"

In [175]:
perc_joint = total_mort_ppl/total_ppl
perc_joint.index = ['Unknown',
                    '18–24',
                    '25–34',
                    '35-44',
                    '45-54',
                    '55–64',
                    '65–74',
                    '75+'] 

perc_joint.index = perc_joint.index + "_perc_joint"

In [176]:
mort_per_capita = new_mort.groupby('consumer_age_archive').sum()['balance']/total_ppl
mort_per_capita.index = ['Unknown',
                    '18–24',
                    '25–34',
                    '35-44',
                    '45-54',
                    '55–64',
                    '65–74',
                    '75+'] 

mort_per_capita.index = mort_per_capita.index + "_mort_per_capita"

In [185]:
row = pd.concat([perc_mort, perc_joint, mort_per_capita, finalstat])
row

,0
Unknown_mort_perc,0.669550
18–24_mort_perc,0.564372
25–34_mort_perc,0.599753
35-44_mort_perc,0.639210
45-54_mort_perc,0.653657
55–64_mort_perc,0.666171
65–74_mort_perc,0.681523
75+_mort_perc,0.650927
Unknown_perc_joint,0.216804
18–24_perc_joint,0.025643


# Percentage of people who have EVER had a mortgage, by age

I think the best way to do this is to use the "first_c" column in balances in and out 

In [3]:
biao_cols = ['first_c']
biao_str = output_3 + "1_balance_in_and_out.parquet"

biao = pd.read_parquet(biao_str, columns = biao_cols).reset_index()

In [10]:
biao_mort = biao[biao.product_category == "FM"].drop(['trade_id', 'product_category'], axis = 1)

In [13]:
biao_mort = biao_mort.sort_values('first_c').drop_duplicates('consumer_id', keep = 'first')

In [37]:
monnum = 175

# read in a month
cols = ['consumer_id', 'consumer_age_archive']

month_str = sample + "202001_10perc.parquet"
mon = pd.read_parquet(month_str, columns = cols).drop_duplicates()

In [34]:
biao_mort_temp = biao_mort[biao_mort.first_c <= monnum].consumer_id

In [42]:
total = mon.groupby('consumer_age_archive').count()

In [43]:
group = mon[mon.consumer_id.isin(biao_mort_temp)].groupby('consumer_age_archive').count()

In [45]:
group/total

,consumer_id
consumer_age_archive,
0,0.290885
1,0.026915
2,0.233730
3,0.512570
4,0.651903
5,0.688616
6,0.645518
7,0.414973
